In [16]:
# import required libraries
import pandas as pd
import numpy as np

In [23]:
#import .csv and assign the variable onlinead, preview first rows
onlinead = pd.read_csv('path_to_file')
onlinead.head(10)

,customerID,test group,made_purchase,days_with_most_add,peak ad hours,ad_count
0,1,ad,False,24,20,5
1,2,psa,False,21,16,9
2,3,psa,False,1,18,8
3,4,ad,False,20,23,7
4,5,ad,False,3,13,5
5,6,ad,False,13,22,7
6,7,ad,False,7,19,6
7,8,psa,False,6,22,10
8,9,psa,False,6,15,7
9,10,psa,False,2,19,5


In [18]:
# 1. Data Preprocessing
# checking for missing values
onlinead.isnull().sum()

customerID            0
test group            0
made_purchase         0
days_with_most_add    0
peak ad hours         0
ad_count              0
dtype: int64

In [35]:
# replace spaces in data with underscores for improved clarity
onlinead.rename(columns=lambda x: x.strip().replace(" ", "_"), inplace=True)
# rename days_with_most_add to days_with_most_ads
onlinead.rename(columns={'days_with_most_add': 'days_with_most_ads'}, inplace=True)
onlinead.head()


,customerID,test_group,made_purchase,days_with_most_ads,peak_ad_hours,ad_count
0,1,ad,False,24,20,5
1,2,psa,False,21,16,9
2,3,psa,False,1,18,8
3,4,ad,False,20,23,7
4,5,ad,False,3,13,5


In [25]:
# 2. Grouping and Aggregating Data
# amount of customers in each test group
onlinead_groups = onlinead['customerID'].groupby(onlinead['test_group']).nunique()
onlinead_groups

test_group
ad     12053
psa     7947
Name: customerID, dtype: int64

In [26]:
# calculate percentage of purchases, for each test group
onlinead_groups_purchases = onlinead.groupby(onlinead['test_group']).agg({'made_purchase': 'mean'}) * 100
onlinead_groups_purchases
#This indicates that the ad group had more than double the purchase rate compared to the PSA group, 
# suggesting that showing ads has a stronger influence on purchase behavior than showing PSAs


,made_purchase
test_group,
ad,6.662242
psa,3.233925


In [34]:
# 3. Ad Exposure Analysis
# Is there a relationship between the amount of ads a user has seen and the likelihood of a purchase? 
# group onlinead by ad_count and calculate mean of made_purchase

onlinead_exposure_combined = onlinead.groupby('ad_count').agg({'made_purchase': 'mean'})
onlinead_exposure_combined
#rename made_purchase to purchase_rate
onlinead_exposure_combined.columns = ['purchase_rate_comb']
onlinead_exposure_combined

# Increased ad exposure slightly increases the likelihood of a purchase, but only up to a certain point (7 ads). In addition to that,
# the pattern is not entirely consistent: the likelihood of a purchase rises when test persons are exposed to two ads, but decreases again 
# when exposed to more than two ads. 

,purchase_rate
ad_count,
1,0.047475
2,0.060697
3,0.056158
4,0.041372
5,0.049372
6,0.051731
7,0.061968
8,0.053825
9,0.050919


In [31]:
# 4. Comparing Ad and PSA Groups
# compare how exposure impacts purchase behavior for ads versus PSAs
onlinead_exposure_separate = onlinead.groupby(['test_group', 'ad_count']).agg({'made_purchase': 'mean'})
onlinead_exposure_separate

#rename made_purchase to purchase_rate
onlinead_exposure_separate.columns = ['purchase_rate']
onlinead_exposure_separate

# convert onlinead_exposure_separate to wide format for increased readability
onlinead_exposure_separate = pd.pivot_table(onlinead_exposure_separate, 
                                           index = 'ad_count', 
                                            columns = 'test_group', 
                                            values = 'purchase_rate')
onlinead_exposure_separate

#  The results indicate that ad exposure does seem to increase the likelihood of a purchase, as the purchase rates of the ad group are 
# consistently higher compared to the PSA group. 

test_group,ad,psa
ad_count,,
1,0.056572,0.033419
2,0.077922,0.033419
3,0.069021,0.035714
4,0.053245,0.023077
5,0.062561,0.029390
6,0.064146,0.032854
7,0.082023,0.032803
8,0.068678,0.031640
9,0.064086,0.030888


In [42]:
# 5. Analyzing Peak Ad Hours
# What is the relationship between the peak ad hours and the likelihood of a purchase? 
# group onlinead by peak_ad_hours and calculate mean of made_purchase

onlinead_peak_comb = onlinead.groupby(['peak_ad_hours']).agg({'made_purchase': 'mean'})
onlinead_peak_comb
#rename made_purchase to purchase_rate
onlinead_peak_comb.columns = ['purchase_rate_comb']
onlinead_peak_comb

,purchase_rate_comb
peak_ad_hours,
0,0.065789
1,0.051282
2,0.046448
3,0.040936
4,0.031662
5,0.081325
6,0.043988
7,0.076705
8,0.053073


In [39]:
# 6. Time of Day Analysis
# As onlinead_peak_combined is hard to read, the individual hours of the day will be combined into broader categories:
# night: 0-5, morning: 6-11, afternoon: 12-17, evening: 18-23

def categorize_hours(peak_ad_hours):
    if peak_ad_hours >= 0 and peak_ad_hours <= 5:
        return 'night'
    elif peak_ad_hours >= 6 and peak_ad_hours <= 11:
        return 'morning'
    elif peak_ad_hours >= 12 and peak_ad_hours <= 17:
        return 'afternoon'
    else:
        return 'evening'

# Apply the function to create a new column
onlinead['time_category'] = onlinead['peak_ad_hours'].apply(categorize_hours)
onlinead.head()


,customerID,test_group,made_purchase,days_with_most_ads,peak_ad_hours,ad_count,time_category
0,1,ad,False,24,20,5,evening
1,2,psa,False,21,16,9,afternoon
2,3,psa,False,1,18,8,evening
3,4,ad,False,20,23,7,evening
4,5,ad,False,3,13,5,afternoon


In [43]:
# What is the relationship between the time of the day and the likelihood of a purchase?
# group onlinead by time_category and calculate mean of made_purchase

onlinead_timecat_comb = onlinead.groupby(['time_category']).agg({'made_purchase': 'mean'})
onlinead_timecat_comb
#rename made_purchase to purchase_rate
onlinead_timecat_comb.columns = ['purchase_rate_comb']
onlinead_timecat_comb

,purchase_rate_comb
time_category,
afternoon,0.050561
evening,0.055313
morning,0.049006
night,0.052558


In [49]:
# 7. Combining Time and Ad Exposure
# What is the relationship between the purchase rate, the time of the day and the amount of ads customers have seen (within the ads group)?
# Filter onlinead for the ads group
onlinead_ad = onlinead[onlinead['test_group'] == 'ad']

# group onlinead_ad by time_category and ad_count and calculate mean of made_purchase
onlinead_timecat_adcount = onlinead_ad.groupby(['time_category', 'ad_count']).agg({'made_purchase': 'mean'})
onlinead_timecat_adcount


made_purchase
time_category ad_count               
afternoon     1              0.061728
              2              0.056180
              3              0.089888
              4              0.038462
              5              0.068611
              6              0.058480
              7              0.072805
              8              0.060362
              9              0.057915
              10             0.067073
evening       5              0.055614
              6              0.067149
              7              0.086345
              8              0.072573
              9              0.067108
              10             0.062245
morning       1              0.043796
              2              0.097561
              3              0.054688
              4              0.061594
              5              0.054902
night         1              0.069106
              2              0.067616
              3              0.075540
              4              0.048583
              5              0.081181

In [50]:
# convert onlinead_timecat_adcount_comb to wide format for improved readability

onlinead_timecat_adcount = pd.pivot_table(onlinead_timecat_adcount, 
                                              index = 'ad_count', 
                                               columns = 'time_category',
                                               values = 'made_purchase', 
                                              )
onlinead_timecat_adcount
# The results indicate varying purchase rates across different time categories and levels of ad exposure. The only value for which data 
# across all time categories is available is the exposure to five ads and this level of exposure is most effective at night.

time_category,afternoon,evening,morning,night
ad_count,,,,
1,0.061728,NaN,0.043796,0.069106
2,0.056180,NaN,0.097561,0.067616
3,0.089888,NaN,0.054688,0.075540
4,0.038462,NaN,0.061594,0.048583
5,0.068611,0.055614,0.054902,0.081181
6,0.058480,0.067149,NaN,NaN
7,0.072805,0.086345,NaN,NaN
8,0.060362,0.072573,NaN,NaN
9,0.057915,0.067108,NaN,NaN
